# VARify


## Workflow


<img src="wf.png" width="600">

## Exploring Data

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

# mpileup 
mp = pd.read_table('../data/test/mpiletest-out.txt', header=None)
mp = mp.rename(columns={1:"snp_pos", 4:"pileup"})

# custom out file
cf = pd.read_table('../data/test/snp_out.txt')

In [2]:
# column names
# cols = ['chr_id', 'snp_pos', 'ref_allele', 
#         'alt_allele', 'gene_id', 'mrna_id',
#         'prot_id', 'strand', 'effect', 
#         'snp_cds_pos', 'codon1_genome_pos',
#         'codon2_genome_pos', 'codon3_genome_pos', 
#         'snp_aa_pos', 'ref_codon','alt_codon', 
#         'ref_aa', 'alt_aa']

sub_cols = ['chr_id', 'snp_pos', 'ref_allele', 
            'alt_allele', 'codon1_genome_pos',
            'codon2_genome_pos', 'codon3_genome_pos', 
            'ref_codon']

# subset of custom file, cf
sub_cf = cf[sub_cols]
sub_cf

,chr_id,snp_pos,ref_allele,alt_allele,codon1_genome_pos,codon2_genome_pos,codon3_genome_pos,ref_codon
0,chr1,521,G,C,519,520,521,AAG
1,chr1,523,A,T,521,522,523,GTA
2,chr1,524,C,G,522,523,524,TAC


In [3]:
mp

,0,snp_pos,2,3,pileup,5,6
0,seq1,521,G,26,".,......A.a.A,..,..,.,.,,^].",DJJJJJJJJJJJJJJJJJJJJJJJA7,]]]]]]]]]]]]]]]]]]]]]]]]]]
1,seq1,522,T,25,",...........,..,..,.,.,,.",JJJJJJJJJJJJJJJJJJJJJJJA7,]]]]]]]]]]]]]]]]]]]]]]]]]
2,seq1,523,A,25,tTTTTTTTTTTTtTTtTTtTtTttT,JJJJJJJJJJJJJJJJJJJJJJJA7,]]]]]]]]]]]]]]]]]]]]]]]]]
3,seq1,524,C,27,",...G.......,.G,..,.,.,,G..",JJJJJJJJJJJJJJJJJJJJJJJJG88,]]]]]]]]]]]]]]]]]]]]]]]]]]]


## Merging the two dataframes together

In [4]:
sub_mp = mp[['snp_pos','pileup']]
merged_var = pd.merge(cf, sub_mp, on='snp_pos')
merged_var

,chr_id,snp_pos,ref_allele,alt_allele,gene_id,mrna_id,prot_id,strand,effect,snp_cds_pos,codon1_genome_pos,codon2_genome_pos,codon3_genome_pos,snp_aa_pos,ref_codon,alt_codon,ref_aa,alt_aa,pileup
0,chr1,521,G,C,seq1,seq1.1,seq1.1,+,synonymous,265,519,520,521,89,AAG,AAC,K,N,".,......A.a.A,..,..,.,.,,^]."
1,chr1,523,A,T,seq1,seq1.1,seq1.1,+,synonymous,267,521,522,523,89,GTA,GTT,E,E,tTTTTTTTTTTTtTTtTTtTtTttT
2,chr1,524,C,G,seq1,seq1.1,seq1.1,+,synonymous,268,522,523,524,89,TAC,TAG,Y,X,",...G.......,.G,..,.,.,,G.."


## Split Dataframe

Splitting the snpeff table by codons, get genome position and alt/ref codon



In [27]:
# Utility File

# libraries
import warnings
import pandas as pd

# For every row in here, we're going to apply this uniqueSNP function

def mergeTab(mp, so):
    sub_mp = mp[['snp_pos', 'pileup']]
    mdf = pd.merge(so, sub_mp, on='snp_pos')
    return mdf


def varOnly(a):
    '''
    purpose: remove non-nt values in pileup
    input: string of mpileup 
    output: nt string
    '''
    x = [i for i in a if i.upper() in ['A','C','T','G']]
    
    return x

def isMono(ref, temp):
    # Check if two snps are monomorphic
    c=0
    count = [c+1 for snp in temp if snp == ref]
    if sum(count) >= 2: 
        warnings.warn("More than 2 monomorphic alleles at position.")
        return True
    else:
        return False
    
def multiError(x, pos):
    if len(x) > 2: 
        warnings.warn("There are more than 2 SNPS at position"); print(pos)
    
def uniqueSNP(x):
    '''
    Creates a combination of SNPs
    Replaces Alt at each position
    '''
    l = str(x['pileup'])
    # make upper
    l = l.upper() # just to make simple
    l = varOnly(l)

    # Get list of unique
    a = [i for i in l]
    a_unique = list(set(a)) 
    
    # create a dictionary
    df = pd.DataFrame()
    temp = [i for i in x['ref_codon']]
    snp_dict = {}
    
#     flag = False
    
    multiError(a_unique, x['snp_pos'])

    test = []
    for snp in a_unique:
        snp_dict['snp_pos'] = int(x['snp_pos'])
        snp_dict['varify_alt'] = snp

#         if flag:
#             snp_dict['flag'] = '*'
#         else:
#             snp_dict['flag'] = ''
        
        for i in range(len(temp)):
            temp[i] = snp
            test.append(''.join(temp))
            snp_dict[f"varify_codon{i+1}"] = ''.join(temp)
            temp = [i for i in ref]
            
        flag = isMono(ref, test)  
        
        if flag: 
            snp_dict['flag'] = '*'
        else:
            snp_dict['flag'] = ''
            
        df = df.append(snp_dict, ignore_index=True)


        snp_dict = {}
        test = []

    df = pd.merge(cf, df, on='snp_pos')
    return df

def altValidate(x):
    # Varify alt == Alt? T or F
    # Add flag to capture
    
    x['VARified'] = 'Yes' if x['alt_allele'] == x['varify_alt'] else 'No'
    
    return x

def varify(x):
    df = x.apply(axis=1, func=uniqueSNP)
    df = pd.concat(list(df))
    df = df.apply(axis=1, func=altValidate)
    reorg = ['chr_id', 'snp_pos', 'ref_allele', 'alt_allele', 
             'varify_alt', 'VARified', 
             'codon1_genome_pos','codon2_genome_pos',
             'codon3_genome_pos', 'ref_codon', 'varify_codon1', 
             'varify_codon2', 'varify_codon3','flag']
    
    return df[reorg]

In [28]:
mdf = mergeTab(mp, cf)
varify(merged_var)

,chr_id,snp_pos,ref_allele,alt_allele,varify_alt,VARified,codon1_genome_pos,codon2_genome_pos,codon3_genome_pos,ref_codon,varify_codon1,varify_codon2,varify_codon3,flag
0,chr1,521,G,C,A,No,519,520,521,AAG,AAG,AAG,AAA,*
0,chr1,523,A,T,T,Yes,521,522,523,GTA,TTA,GTA,GTT,
0,chr1,524,C,G,G,Yes,522,523,524,TAC,GAC,TGC,TAG,


In [13]:
# pile up 
mp

,0,snp_pos,2,3,pileup,5,6
0,seq1,521,G,26,".,......C.T.A,..,..,.,.,,^].",DJJJJJJJJJJJJJJJJJJJJJJJA7,]]]]]]]]]]]]]]]]]]]]]]]]]]
1,seq1,522,T,25,",...........,..,..,.,.,,.",JJJJJJJJJJJJJJJJJJJJJJJA7,]]]]]]]]]]]]]]]]]]]]]]]]]
2,seq1,523,A,25,tTTTTTTTTCTTtTTtTCtTtTttC,JJJJJJJJJJJJJJJJJJJJJJJA7,]]]]]]]]]]]]]]]]]]]]]]]]]
3,seq1,524,C,27,",...G.......,.G,..,.,.,,A..",JJJJJJJJJJJJJJJJJJJJJJJJG88,]]]]]]]]]]]]]]]]]]]]]]]]]]]


In [12]:
# snp-eff like table
cf

,chr_id,snp_pos,ref_allele,alt_allele,gene_id,mrna_id,prot_id,strand,effect,snp_cds_pos,codon1_genome_pos,codon2_genome_pos,codon3_genome_pos,snp_aa_pos,ref_codon,alt_codon,ref_aa,alt_aa
0,chr1,521,G,C,seq1,seq1.1,seq1.1,+,synonymous,265,519,520,521,89,AAG,AAC,K,N
1,chr1,523,A,T,seq1,seq1.1,seq1.1,+,synonymous,267,521,522,523,89,GTA,GTT,E,E
2,chr1,524,C,G,seq1,seq1.1,seq1.1,+,synonymous,268,522,523,524,89,TAC,TAG,Y,X


### To Do
* [x] output comparison
* [ ] Type Effect
* [x] Make into Function
* [ ] Build larger test data
* [ ] Nextflow with Assembly/Sam to mpileup


### Notes
**Test Data:**
* Take processed mRNA transcript (FASTA) —> VCF w/ 2-3 SNPS

**Process:** 
* RNA FASTA -align to genome using gmap->SAM/BAM—>BAM -variantcalling-> VCF —> Capture Variants
* DNA FASTA -align to genome using gmap->SAM/BAM—>BAM -variantcalling-> VCF —> Capture Variants |—> Compare RV and GV

# VARify for same ALT as prediction

| Pos     | Ref     | Alt |Varify_Alt|Varify|
| ----------- | ----------- | ----------- | ----------- | ----------- |
|202| A      | T       |T|Yes|
|204| C   | G        |.|No|


In [29]:
varify_out = varify(merged_var)

varify_out.iloc[:,:6]


,chr_id,snp_pos,ref_allele,alt_allele,varify_alt,VARified
0,chr1,521,G,C,A,No
0,chr1,523,A,T,T,Yes
0,chr1,524,C,G,G,Yes


In [31]:
varify_out.loc[varify_out['VARified'] == 'Yes']

,chr_id,snp_pos,ref_allele,alt_allele,varify_alt,VARified,codon1_genome_pos,codon2_genome_pos,codon3_genome_pos,ref_codon,varify_codon1,varify_codon2,varify_codon3,flag
0,chr1,523,A,T,T,Yes,521,522,523,GTA,TTA,GTA,GTT,
0,chr1,524,C,G,G,Yes,522,523,524,TAC,GAC,TGC,TAG,


# Capture codon combinations from different positions

Logic: 
If position 1,2,3 and 1 and 3 have variances - we want to maintain these variances. 

We can place variances of 1 and 3 into dictionaries. 
If at position 3 and position 1 has variances --> combine and showcase AA



In [102]:
# Codon Combos

def altDict(x):
    '''
    
    Creates a combination of SNPs
    if snps have existing alts
    
    Outputs into a dictionary
    
    '''
    # At each line, takes reference codon
    sub_x = x.loc[x['VARified'] == 'Yes']
    
    # create a dictionary of Alts
    alt_dict = dict(zip(sub_x['snp_pos'], sub_x['alt_allele']))
    
    return alt_dict

def altCombo(x, alt_dict=alt_dict):
    '''
    Creates combos based on
    Current position and alt dictionary
    Can be used with apply
    '''
    comboDict = {}
    p3 = int(x['snp_pos'])
    p2 = p3-1
    p1 = p2-1
    
    comboDict = {p1 : '.', p2 : '.', p3 : '.'}
    
    # create a dictionary
    df = pd.DataFrame()
    temp = [i for i in x['ref_codon']]
    snp_dict = {}
    
    res = {key: alt_dict.get(key, comboDict[key]) for key in comboDict}
    ''.join(res.values())
    
    # make combinations
    for i in range(len(temp)):
        temp[i] = snp
        test.append(''.join(temp))
        snp_dict[f"varify_codon{i+1}"] = ''.join(temp)
        temp = [i for i in ref]

    flag = isMono(ref, test)  

    if flag: 
        snp_dict['flag'] = '*'
    else:
        snp_dict['flag'] = ''

    df = df.append(snp_dict, ignore_index=True)


    snp_dict = {}
    test = []

    df = pd.merge(cf, df, on='snp_pos')
    
    return comboDict


def flatten(d, parent_key='', sep='_'):
    import collections
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [111]:
# At each line, takes reference codon
sub_x = varify_out.loc[varify_out['VARified'] == 'Yes']

# ref = sub_x['ref_codon']
alt_dict = dict(zip(sub_x['snp_pos'], sub_x['alt_allele']))

alt_dict

combo_dict = varify_out.apply(axis = 1, func=altCombo)
# combo_dict = flatten(combo_dict)

res = {key: alt_dict.get(key, combo_dict[key]) for key in combo_dict}
#res = {key:value for (key, value) in res.items() if value is not None}
res.items

KeyError: "None of [Int64Index([519, 520, 521], dtype='int64')] are in the [index]"

'TG'

# Running Script

In [ ]:
%reload_ext autoreload
%autoreload 2

%run '../script/main.py' --in_mp '../data/test/mpiletest-out.txt' --in_so '../data/test/snp_out.txt'